In [1]:
import pandas as pd
import pickle
from collections import Counter
import math
import nltk
import requests
import html
import datetime
from datetime import timedelta, date
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import Lasso

%matplotlib inline

In [2]:
data = pd.read_csv("../../dados/resultado.csv", header = None) 

* Data da mensagem
* Usuário que postou a mensagem
* Texto da mensagem
* Pontuação para  classe=paródia
* Pontuação para  classe=opinião
* Pontuação para  classe=campanha
* Pontuação para  classe=experiência pessoal
* Pontuação para  classe=informação
* Resultado (qual a classe foi escolhida pela maior pontuação)
* Informação (JSON) sobre a localização

In [3]:
data.columns = ["date", "user", "message", "0", "1", "2", "3", "4", "class", "location"]

In [4]:
data['date'] = pd.to_datetime(data['date'])

In [5]:
data.dtypes

date        datetime64[ns]
user                object
message             object
0                  float64
1                  float64
2                  float64
3                  float64
4                  float64
class                int64
location            object
dtype: object

In [6]:
data = data[data['date'] < '2011-09-30 23:59:59']

# save pickle of filtered data

# load pickle

In [16]:
with open('D:/Dropbox/A-Marcelo/education-work/2019-ufmg/resultado.pkl', 'rb') as f:
    data = pickle.load(f)

In [17]:
data = data[data['date'] < '2011-09-30 23:59:59']

In [18]:
data['date'] = data['date'].dt.strftime("%Y-%m-%d")

In [19]:
data['date'] = pd.to_datetime(data['date'])

In [20]:
data.tail()

,date,user,message,0,1,2,3,4,class,location
738140,2011-09-12,claricereis91,"@berntfinal Dengue num dá dor na garganta, vc ...",0.147137,0.066065,0.069753,0.544475,0.172570,0,"{u'd': 1460387911967, u'v': 1}"
738141,2011-09-12,guinisinmypants,@DearDany Todos os sintomas de dengue.. Até do...,0.130242,0.059683,0.119528,0.488431,0.202117,0,"{u'd': 1460387911968, u'v': 1}"
738142,2011-09-12,eu_plfl,a @pqjade_ tem problemas e dengue no #cerebro,0.295350,0.144311,0.110989,0.201095,0.248256,1,"{u'city': u'Sao Jose dos Pinhais', u'country':..."
738143,2011-09-12,Pathy_Aleixo,@ronaldoarreios Tô com dengue.... Meu corpo da...,0.084619,0.036423,0.048278,0.739830,0.090849,0,"{u'd': 1460387912416, u'v': 1}"
738144,2011-09-12,_rachelmarques,Minha ta mais parada do que foco de mosquito d...,0.500673,0.088380,0.053413,0.200705,0.156830,1,"{u'city': u'Rio de Janeiro', u'country': u'Bra..."


In [21]:
data.shape

(603588, 10)

### stopwords

In [22]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords = list(set(stopwords))

In [23]:
try: stopwords.remove('não')
except: pass

In [24]:
def remove_stopwords(text):
    tokens = []
    for word in text.lower().split():
        if word not in stopwords:
            tokens.append(word)
    return tokens

new column: tokens without stopwords

In [25]:
data['message_nostopwords'] = data['message'].apply(remove_stopwords)

### build epidemiologic weeks
* Week one of every year is the first week with 4 days or more.
* Timestamp of the study comprises: 21/11/2010-30/09/2011

In [26]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2010, 1, 1)
end_dt = date(2011, 10, 30)
dates = []
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))

In [27]:
dates_2010 = dates[:365]
dates_2011 = dates[365:]

In [28]:
ep_week_lst = []
count = 0
epidemic_week = 1
ep_week_lst.extend([201000]*2)
for d in dates_2010[2:]:
    if count == 7:
        epidemic_week += 1
        count = 0
    count += 1
    str_week = str(epidemic_week)
    if len(str_week) == 1: str_week = '0'+str_week
    year_week = '2010'+str_week
    ep_week_lst.append(year_week)
    
count = 0
epidemic_week = 1
ep_week_lst.append(201100)
for d in dates_2011[1:]:
    if count == 7:
        epidemic_week += 1
        count = 0
    count += 1
    str_week = str(epidemic_week)
    if len(str_week) == 1: str_week = '0'+str_week
    year_week = '2011'+str_week
    ep_week_lst.append(year_week)

In [29]:
datetime.datetime.strptime("21/12/2008 20:04:44", "%d/%m/%Y %H:%M:%S").strftime("%Y-%m-%d")

'2008-12-21'

In [30]:
data.dtypes

date                   datetime64[ns]
user                           object
message                        object
0                             float64
1                             float64
2                             float64
3                             float64
4                             float64
class                           int64
location                       object
message_nostopwords            object
dtype: object

In [31]:
def date_to_epweek(row):
    #print(type(row))
    #print(row)
    row = row.strftime("%Y-%m-%d")
    date_index = dates.index(row)
    year_week = ep_week_lst[date_index]
    return year_week
data['se'] = data['date'].apply(date_to_epweek) #se = epidemiologic week

In [42]:
data.head()

,date,user,message,0,1,2,3,4,class,location,message_nostopwords,se
0,2010-11-19,PauloAfBahia,A dengue mata mas com simples gestos de preven...,0.138010,0.101521,0.254248,0.085880,0.420341,3,"{u'd': 1443198862750, u'v': 1}","[dengue, mata, simples, gestos, prevenção, pod...",201046
1,2010-11-19,PortalOuroNegro,Mutirão contra a Dengue começou nesta sexta em...,0.105452,0.056434,0.248333,0.188912,0.400869,3,"{u'city': u'Linhares', u'country': u'Brasil', ...","[mutirão, contra, dengue, começou, nesta, sext...",201046
2,2010-11-19,Kustelaas,@depChicoAlencar amannhã alem do dia da consiê...,0.054194,0.084122,0.213530,0.146927,0.501226,3,"{u'country': u'Brasil', u'state': u'MT', u'typ...","[@depchicoalencar, amannhã, alem, dia, consiên...",201046
3,2010-11-19,Raul_Jungmann,Pernambuco tem risco de sofrer epidemia de den...,0.080289,0.106928,0.054929,0.141276,0.616577,3,"{u'city': u'Recife', u'country': u'Brasil', u'...","[pernambuco, risco, sofrer, epidemia, dengue, ...",201046
4,2010-11-19,laana_mello,Rpz...acho q a Caueira tah com surto de dengue...,0.112013,0.143425,0.094947,0.429626,0.219990,0,"{u'country': u'Brasil', u'state': u'MT', u'typ...","[rpz...acho, q, caueira, tah, surto, dengue,, ...",201046


# Infodengue
The research comprises 12 cities:
* Belém, Belo Horizonte, Brasília,Fortaleza, Manaus, Natal, Rio Branco, Rio de Janeiro, São Paulo, Londrina, Niterói, Ribeirão Preto.
* Source of geocodes: https://ww2.ibge.gov.br/home/geociencias/geografia/redes_fluxos/gestao_do_territorio_2014/base.shtm
* File of the geocodes: Base_de_dados_dos_municipios.xls

### geocodes

In [33]:
geocodes_BR = pd.read_excel (r'..\..\dados\Base_de_dados_dos_municipios.xls')
geocodes_BR = geocodes_BR[['UF', 'Codmundv', 'NomeMunic']]

### infodengue - Rio de Janeiro

In [34]:
municipality= 'Rio de Janeiro'.upper()
geocode_row = geocodes_BR.loc[geocodes_BR['NomeMunic'] == municipality]
geocode = list(geocode_row.iloc[:,1])[0]
url = 'https://info.dengue.mat.br/api/alertcity/?geocode=3304557&disease=dengue&format=csv&ew_start=47&ey_start=2010&ew_end=39&ey_end=2011'
r = requests.get(url)
if r:
    print('Success!')
else:
    print('An error has occurred.')

Success!


In [35]:
r.headers

{'Server': 'nginx/1.4.6 (Ubuntu)', 'Date': 'Thu, 11 Apr 2019 10:49:09 GMT', 'Content-Type': 'text/plain', 'Content-Length': '11553', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Vary': 'Accept-Language, Cookie, Accept-Encoding', 'Content-Language': 'pt-br', 'Content-Encoding': 'gzip'}

In [36]:
r = r.text.split('\n')

In [37]:
count = 0
for row in r:
    split_row = row.split(',')
    r[count] = split_row
    count += 1
headers = r.pop(0)

In [38]:
df = pd.DataFrame(r, columns=headers)
df = df.iloc[:,2:6]
df.head()

,se,data,tweets,casos
0,201135,2011-08-28,,12
1,201112,2011-03-20,,530
2,201104,2011-01-23,,56
3,201123,2011-06-05,,29
4,201049,2010-12-05,,6


In [39]:
df.shape

(451, 4)

In [40]:
df['casos'] = df['casos'].fillna(0).astype(int)

In [41]:
df.groupby(['se']).sum().head(3)

,casos
se,
201047,74
201048,93
201049,149


### check epidemiologic week consistency

In [117]:
df.loc[df['data'] == '2011-01-16']

,se,data,tweets,casos
11,201103,2011-01-16,,8
35,201103,2011-01-16,,8
49,201103,2011-01-16,,14
78,201103,2011-01-16,,33
113,201103,2011-01-16,,16
157,201103,2011-01-16,,28
235,201103,2011-01-16,,27
320,201103,2011-01-16,,48
388,201103,2011-01-16,,25
437,201103,2011-01-16,,30


### group tweet data by week

In [ ]:
data.groupby(['se']).sum()

### TF-IDF

https://www.kdnuggets.com/2018/08/wtf-tf-idf.html

In [ ]:
tfidf = TfidfVectorizer(
    analyzer='word')  

In [ ]:
tfidf.fit(data['message_nostopwords'])

In [ ]:
idf_counter = Counter()
for text in data['message_nostopwords']:
    #words = text.lower().split()
    #idf_counter.update(words)
    idf_counter.update(text)

In [ ]:
idf = {
  word: math.log(len(data)/df, 2)
  for word, df in idf_counter.items()
}

In [ ]:
def analyze_row(row):
    target_words = row.lower().split()
    tfidf =  {
        word: (1 + math.log(_tf, 2)) * idf[word]
        for word, _tf in Counter(target_words).items()
    }
    num_words = 20
    most_frequent = [
        w  for (w, _) in Counter(target_words).most_common(num_words)
    ]
    sorted_tfidf = [
        w  for (w, _) in sorted(tfidf.items(), key=lambda kv: kv[1], reverse=True)
    ]
    print(row)
    print("Most frequent: {}".format(most_frequent))
    print("Higher TF-IDF: {}".format(sorted_tfidf[:num_words]))

In [ ]:
idf_top = sorted(idf, key=idf.get)[:30]

In [ ]:
for term in idf_top:
    data[term] = 0

In [ ]:
def tfidf(text, term):
    tfidf = 0
    if term in text:
        tf = idf_counter[term]
        tfidf = (1 + math.log(tf, 2)) * idf[term]
        return tfidf
    return tfidf

In [ ]:
for term in idf_top:
    data[term] = data['message'].apply(lambda text: tfidf(text, term))

In [ ]:
sorted(idf, key=idf.get)[-4:]

# To do

### Lasso regression
Using Lasso to choose the most relevant terms

In [86]:
data.head()

,date,user,message,0,1,2,3,4,class,location,...,q,saúde,to,água,pra,dengue!,parada,combater,-,hoje
0,19/11/2010 20:04:44,PauloAfBahia,A dengue mata mas com simples gestos de preven...,0.138010,0.101521,0.254248,0.085880,0.420341,3,"{u'd': 1443198862750, u'v': 1}",...,0.000000,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19/11/2010 20:05:05,PortalOuroNegro,Mutirão contra a Dengue começou nesta sexta em...,0.105452,0.056434,0.248333,0.188912,0.400869,3,"{u'city': u'Linhares', u'country': u'Brasil', ...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19/11/2010 20:05:30,Kustelaas,@depChicoAlencar amannhã alem do dia da consiê...,0.054194,0.084122,0.213530,0.146927,0.501226,3,"{u'country': u'Brasil', u'state': u'MT', u'typ...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19/11/2010 20:05:58,Raul_Jungmann,Pernambuco tem risco de sofrer epidemia de den...,0.080289,0.106928,0.054929,0.141276,0.616577,3,"{u'city': u'Recife', u'country': u'Brasil', u'...",...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19/11/2010 20:06:18,laana_mello,Rpz...acho q a Caueira tah com surto de dengue...,0.112013,0.143425,0.094947,0.429626,0.219990,0,"{u'country': u'Brasil', u'state': u'MT', u'typ...",...,28.714269,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
data.columns[11:]

Index(['dengue', 'rt', 'é', 'combate', 'contra', 'dengue.', 'não', 'dia',
       'mosquito', 'casos', 'brasil', 'quase', 'ano', 'registra', 'relação',
       'dengue,', 'dobro', 'número', 'ta', 'd', 'q', 'saúde', 'to', 'água',
       'pra', 'dengue!', 'parada', 'combater', '-', 'hoje'],
      dtype='object')

In [91]:
X = data.iloc[:,11:]

In [93]:
X.head()

,dengue,rt,é,combate,contra,dengue.,não,dia,mosquito,casos,...,q,saúde,to,água,pra,dengue!,parada,combater,-,hoje
0,7.906893,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.00000,0.000000,0.0,23.840136,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.00000,21.798437,0.0,0.000000,0.0,0.0,25.643406,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.906893,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7.906893,21.34936,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,28.714269,0.0,28.714269,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = data

#y = dados da saúde

In [ ]:
lasso = Lasso(alpha=0.4, normalize=True)

# Fit the regressor to the data
lasso.fit(X, y)

# Compute and print the coefficients
lasso_coef = lasso.coef_ 
print(lasso_coef)

# Plot the coefficients
plt.plot(range(len(X)), lasso_coef)
plt.xticks(range(len(X)), ['var 0','var 1','var 2'], rotation=60)
plt.margins(0.02)
plt.show()


# draft

In [ ]:
text_test = [['this', 'is', 'a', 'sample'], ['this', 'is', 'another', 'example']]